In [1]:
import random
import json

### Random test cases
No matter **AND ARRAY** is contiguous or not

In [2]:
def generate_test_case(lower_bound: int, upper_bound: int):
    n, k = random.randint(lower_bound, upper_bound), random.randint(lower_bound, upper_bound)
    a = ' '.join(map(str, [random.randint(0, (1 << 12) - 1) for _ in range(n)]))
    return n, k, a

In [3]:
n, k, a = generate_test_case(1, 10)
print(n, k)
print(a)

10 3
1404 1553 307 3258 1303 3495 482 748 3617 2560


In [4]:
for i in list(map(int, a.split())):
    print(i)

1404
1553
307
3258
1303
3495
482
748
3617
2560


### Precise test cases

Special case where there's **one column** is all **1**

In [5]:
def generate_ones_column(lower_bound: int, upper_bound: int, pos=None):
    n, k = random.randint(lower_bound, upper_bound), random.randint(lower_bound, upper_bound)
    a = [random.randint(0, (1 << 12) - 1) for _ in range(n)]
    pos = random.randint(0, len(bin(max(a))) -3) if pos is None else pos
    a = ' '.join(map(str, [(1 << pos) | x for x in a])) # convert bit at pos to 1
    return n, k, a, pos

In [6]:
n, k, a, pos = generate_ones_column(1, 10)

In [7]:
print(n, k)
print(pos)
for num in [bin(int(x)) for x in a.split()]:
    print(num)

5 1
11
0b101111111110
0b100110110110
0b110101111101
0b101111110011
0b101110010110


The biggest test: array size is up to **2 * 10<sup>4</sup>**, **12** number having **one zero bit** in **distinct position** and the **rest is max value (2<sup>12</sup> - 1)**

In [8]:
def generate_giant():
    MAX_NUM = (1 << 12) - 1
    MAX_BIT = 12
    n, k = MAX_NUM, MAX_NUM
    a = [MAX_NUM for _ in range(n)]
    for i in range(MAX_BIT - 1):
        a[i] = a[i] & (~(1 << i))
    a[-1] = a[-1] & (~(1 << 11))
    return n, k, ' '.join(map(str, a))

In [9]:
n, k, a = generate_giant()

In [10]:
a[:100], a[-4:]

('4094 4093 4091 4087 4079 4063 4031 3967 3839 3583 3071 4095 4095 4095 4095 4095 4095 4095 4095 4095 ',
 '2047')

### Some hand-desgined test cases

If there exists number of elements **less than k** taken **AND** operator together **equals 0**, **no need** to search for **more elements**

In [11]:
n1, k1 = 5, 4
a1 = [12, 3, 14, 15, 1] # (1100, 0011, 1110, 1110, 0001)

In [12]:
n2, k2 = 5, 5
a2 = [14, 13, 11, 7, 15] # (1110, 1101, 1011, 0111, 1111)

If there exists number of elements **greater than k** taken **AND** operator together **equals 0**, **cannot** make **conclusion**

In [13]:
n3, k3 = 5, 2
a3 = [14, 13, 11, 3, 7] # (1110, 1101, 1011, 0011, 0111)

### Let's put things together

In [14]:
from collections import namedtuple
Constants = namedtuple("Constant", ['MAX_NUM', 'MAX_BIT'])
constants = Constants(2**12 - 1, 12)

def count_bit_zero_of_remainder(and_array: int, x: int) -> int:
    count_bit_zero = 0
    for bit in range(constants.MAX_BIT):
        if (1 << bit) & and_array and (1 << bit) & x == 0:
                count_bit_zero += 1
    return count_bit_zero



def get_result(a: list, k: int) -> str:
    max_num =  constants.MAX_NUM
    count_ai = 0

    for bit in range(constants.MAX_BIT):
        if (1 << bit) & max_num:
            most_zeros, num_with_most_zeros = 0, None

            for ai in a:
                if (1 << bit) & ai == 0:
                    temp = count_bit_zero_of_remainder(max_num, ai)
                    
                    if temp > most_zeros:
                        most_zeros = temp
                        num_with_most_zeros = ai
                        
            if most_zeros == 0:
                return "NO"
            count_ai += 1
            max_num &= num_with_most_zeros
            
            if count_ai <= k and max_num == 0:
                return "YES"
    return "NO"

n, k = map(int, input().split())
a = list(map(int, input().split()))

print(get_result(a, k))

3 2
5 4 3
YES


In [15]:
with open('input.txt', 'a') as rf:
    with open('output.txt', 'a') as wf:
        for n, k, a in zip([n1, n2, n3], [k1, k2, k3], [a1, a2, a3]):
            rf.write(f"{n} {k}\n{' '.join(map(str, a))}\n")
            wf.write(get_result(a, k) + '\n')

In [16]:
with open('input.txt', 'a') as rf:
    with open('output.txt', 'a') as wf:
        for i in range(20):
            n, k, a = generate_test_case(0, 100) # low upperbound
            rf.write(f"{n} {k}\n{a}\n")
            wf.write(get_result(list(map(int, a.split())), k) + '\n')
            
        for i in range(20):
            n, k, a = generate_test_case(0, (1 << 12) - 1) # high upperbound
            rf.write(f"{n} {k}\n{a}\n")
            wf.write(get_result(list(map(int, a.split())), k) + '\n')

In [17]:
with open('input.txt', 'a') as rf:
    with open('output.txt', 'a') as wf:
        for i in range(10):
            n, k, a, _ = generate_ones_column(1, 100) 
            rf.write(f"{n} {k}\n{a}\n")
            wf.write(get_result(list(map(int, a.split())), k) + '\n')

        
        n, k, a = generate_giant()
        rf.write(f"{n} {k}\n{a}\n")
        wf.write(get_result(list(map(int, a.split())), k) + '\n')